# Uavsar Data Products

Uavsar has a variety of different type of images:

[Repeat Pass Interferometric](https://uavsar.jpl.nasa.gov/science/documents/rpi-format.html) images contain:
```{admonition} InSAR Data Types
:class: InSAR Data Types
- ANN file (.ann): a text annotation file with metadata
- AMP files (.amp1 and .amp2): amplitude products for flight 1 and flight 2
- COR files (.cor): coherence a measure of the noise level of the phase
- INT files (.int): wrapped phase difference between the two images
- UNW files (.unw): unwrapped phase difference between the two images
- INC files (.inc): incidence angle in radians
- HGT file  (.hgt): the DEM that was used in the InSAR processing
```

Uavsar repeat pass interferometry uses two images of the same place but separated in time. It then uses the change in phase between the first image and the second to infer changes that occurred in that scene between the two image passes. These phase changes are due to either the wave traveling a longer distance (ground movement or refraction) or change wave speeds (atmospheric water vapor and snow).

[Polarimetric PolSAR](https://uavsar.jpl.nasa.gov/science/documents/polsar-format.html) images contain:
- ANN file (.ann): a text annotation file with metadata
- Polsar file (_HHVV_.grd): all the rest of the files will be a pair of polarizations pushed together

Polsar files have a pair of polarizations (VV, VH, HV, HH) combined in their file name. These files are the phase difference between polarization XX and polarization YY. For instance HHHV is the phase difference between HH and HV polarizations. HVVV is the phase difference between HV and VV and so one. There are 6 of these pairs since order is irrelevant. These 6 images are combined to calculate various metrics that tell you about the types of scattering occurring.


[Polarimetric PolSAR](https://uavsar.jpl.nasa.gov/science/documents/polsar-format.html)
- GRD files (.grd): products projected to the ground in geographic coordinates (latitude, longitude)
Finally all images can be in radar slant range or projected into WGS84. Images that have already been projected to ground range will have the extension .grd appended to their file type extension. 

For instance a image of unwrapped phase that has not been georefenced would end with .unw, while one that was georeferenced would end with .unw.grd. You will generally want to use .grd files for most analysis.


### Import Libraries

In [ ]:
try:
    from uavsar_pytools import UavsarScene
except ModuleNotFoundError:
    print('Install uavsar_pytools with `pip install uavsar_pytools`')

from os.path import join, basename
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import holoviews as hv
import rioxarray as rxa
from bokeh.plotting import show
import datashader as ds
from datashader.mpl_ext import dsshow
hv.extension('bokeh', logo=False)

## Interferometric Imagery

explanation of insar data types

In [ ]:
donner_url = 'https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/donner_03904_20002-003_20009-026_0019d_s01_L090_01_int_grd.zip'
work_dir = '/tmp/slurm_tutorial/'

In [ ]:
scene = UavsarScene(work_dir=work_dir, url = donner_url)
scene.url_to_tiffs()

In [ ]:
imgs ={}
for i, fp in enumerate(glob(join(work_dir, basename(donner_url).split('.')[0], '*.tiff'))):
    name = fp.split('.')[-3]
    imgs[name] = rxa.open_rasterio(fp, parse_coordinates=True, default_name = name).rio.reproject('EPSG:3857')

## What topographic features seem to impact coherence?

Investigate with partner:
- lakes
- aspect (south vs north, east vs west)
- elevation
- trees
- roads
- others?

In [ ]:
tiles = hv.element.tiles.EsriUSATopo().opts()
n = 5 # decimate by a factor of 5
cor = hv.Image(hv.Dataset(imgs['cor'][0,::n,::n], kdims=['x','y'])).opts(cmap = 'gray', colorbar=True, xaxis = None, yaxis = None, title = 'Coherence')
hgt = hv.Image(hv.Dataset(imgs['hgt'][0,::n,::n], kdims=['x','y'])).opts(cmap = 'terrain', colorbar=True, xaxis = None, yaxis = None, title= 'DEM', alpha = 0.4)
hgt_trans = hv.Image(hv.Dataset(imgs['hgt'][0,::100,::100], kdims=['x','y'])).opts(alpha = 0, xaxis = None, yaxis = None, title = 'Topo')
cor_tile = tiles  * cor
hgt_tile = tiles  * hgt
imagery = hv.element.tiles.EsriImagery()  * hgt_trans

hv.Layout([cor_tile, hgt_tile, imagery]).opts(width = 400, height = 900)

In [ ]:
import seaborn as sns
xna = imgs['hgt'].data.ravel()
yna = imgs['cor'].data.ravel()
x = xna[(~np.isnan(xna)) & (~np.isnan(yna))][::100]
y = yna[(~np.isnan(xna)) & (~np.isnan(yna))][::100]

df = pd.DataFrame(dict(x=x, y=y))
df['x_cat'] = pd.qcut(df.x, q= 6, precision = 0)
f, ax = plt.subplots(figsize = (12,8))
sns.violinplot(y = df.y[::100], x = df.x_cat[::100], scale = 'count')
plt.xlabel('Elevation Bands (m)')
plt.ylabel('Coherence')

example of coherence imagery with visual to compare

Here we will compare UAVSAR data prodcuts to optical Harmonized Landsat Sentinel-2 (HLS) data.....


example of phase with coherence to see how low coherence effects unwrapping

- roads
- lakes
- avalanche paths

In [ ]:
tiles = hv.element.tiles.EsriUSATopo().opts()
n = 5 # decimate by a factor of 5
cor = hv.Image(hv.Dataset(imgs['cor'][0,::n,::n], kdims=['x','y'])).opts(cmap = 'gray', colorbar=True, xaxis = None, yaxis = None, title = 'Coherence')
unw = hv.Image(hv.Dataset(imgs['unw'][0,::n,::n], kdims=['x','y'])).opts(cmap = 'magma', colorbar=True, xaxis = None, yaxis = None, title= 'Unwrapped Phase', clim = (0, 2*np.pi))
cor_tile = tiles  * cor
unw_tile = tiles  * unw

hv.Layout([cor_tile, unw_tile]).opts(width = 400, height = 900)

two complex images and plot phase and then do complex cross conjugate to see patterns appear

## Polarimetric Imagery

explanation of polarimetric imagery

example of alpha angle and entropy with diagram